# F1 EDA
#### PART 1: 

Here I answer some questions regarding drivers, races, and seasons.

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.giphy.com/media/v1.Y2lkPTc5MGI3NjExNWl6ajhnaDU4aHVteGpmamk2Z3BkdXd3NDFwY3BybG9uMHB2MzZtcSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/sA50TO4Ew7QXu/giphy.gif")

## 1) Importing Packages and Data

In [2]:
import pandas as pd
import numpy as np
import F1_Preprocessing #Internal package to preprocess data
np.random.seed(1994)

In [3]:
#The data stored in this path is obtained from the API of https://ergast.com/mrd/. It is continuously updated.
#To update this data please run the file "1_API_Requests_Ergast.py"

path = 'C:/Users/gabri/Dropbox/Gaby/Proyectos/My_Portafolio/F1/Data/'

ResultsDF=pd.read_csv(path+"ResultsDF.csv")
QualifyingDF=pd.read_csv(path+"QualifyingDF.csv")
LapsDF=pd.read_csv(path+"LapsDF.csv")
PitsDF=pd.read_csv(path+"PitsDF.csv")

C:\Users\gabri\AppData\Local\Temp\ipykernel_24088\478565046.py:8: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  LapsDF=pd.read_csv(path+"LapsDF.csv")


## 2) Data Preprocessing

In [4]:
# Apply the data preprocessing for each df using a function stored in the repo. 
# This is done this way so the preprocessing is consistent throughout all notebooks.
ResultsDF=F1_Preprocessing.preprocess_F1results(ResultsDF)
LapsDF=F1_Preprocessing.preprocess_F1laps(LapsDF)
PitsDF=F1_Preprocessing.preprocess_F1pits(PitsDF)

In [7]:
# Merging features between dfs

# Laps and Pits
Laps_Pits=pd.merge(LapsDF,PitsDF[["season","round","driverid","pit_stop_lap_number","pit_stop_time","pit_stop_duration_in_milliseconds","pit_stop_number"]],
                   left_on=["driverid","season","round","current_lap_number"],
                   right_on=["driverid","season","round","pit_stop_lap_number"],how="left")

# We only have information of the pit stops from 2012, so Laps_Pits will have instances from 2012 or later
Laps_Pits=Laps_Pits[Laps_Pits["season"]>=2012]

# Merging with Results df
SelectedResults=ResultsDF[['final_position', 'final_points', 'final_grid', 'final_laps', 'final_status', 'driverid','driver.nationality', 'constructor.constructorid', 'constructor.nationality', 'race_time.millis','fastestlap.rank', 'fastestlap.lap','fastestlap.time.time', 'fastestlap.averagespeed.units','fastestlap.averagespeed.speed', 'season', 'round','fastestlap.time.in_miliseconds', 'season-round','circuit.circuitid_encoded', 'constructor.constructorid_encoded','driverid_encoded']]

Laps_Pits_Results=pd.merge(Laps_Pits,SelectedResults,on=["driverid","season","round"],how="left")


In [8]:
# Adding extra features

#Feature created to indicated if the lap is the fastest or not
Laps_Pits_Results['IsFastestLap']=np.where(Laps_Pits_Results['current_lap_number'] == Laps_Pits_Results['fastestlap.lap'], 1, 0)
Laps_Pits_Results['IsPitStopLap']=np.where(Laps_Pits_Results['current_lap_number'] == Laps_Pits_Results['pit_stop_lap_number'], 1, 0)

In [15]:
Laps_Pits_Results[Laps_Pits_Results.columns[20:]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 317874 entries, 0 to 317873
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   final_position                     317874 non-null  int64  
 1   final_points                       317874 non-null  float64
 2   final_grid                         317874 non-null  int64  
 3   final_laps                         317874 non-null  int64  
 4   final_status                       317874 non-null  object 
 5   driver.nationality                 317874 non-null  object 
 6   constructor.constructorid          317874 non-null  object 
 7   constructor.nationality            317874 non-null  object 
 8   race_time.millis                   184713 non-null  float64
 9   fastestlap.rank                    316228 non-null  float64
 10  fastestlap.lap                     316228 non-null  float64
 11  fastestlap.time.time               3162